In [1]:
# %matplotlib inline

import http.client
import json
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pika
import time
from ipywidgets import interact

# from bokeh.plotting import output_file, figure, show, hplot

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import output_file, figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Rect

from six.moves import zip

output_notebook()

source = ColumnDataSource({'x': [], 'y': [], 'width': [], 'height': [], 'fill_color': []})

Loading BokehJS ...

In [2]:
# backend service connection
baseEndpoint = "localhost:8082" 
basePath = "/2d/"
baseUri = baseEndpoint + basePath

# rabbitMQ connection
credentials = pika.PlainCredentials("user", "password")
parameters = pika.ConnectionParameters('localhost', 5672, '/', credentials)

# setting up exchange and queue

#channel = connection.channel()
#channel.exchange_declare(exchange='BDAPRO', exchange_type='direct')
#
#result = channel.queue_declare(exclusive=True)   |  channel.queue_declare(queue=new_task_id, durable=True, auto_delete=True)   !!!!

#queue_name = result.method.queue

In [3]:
MESSAGE_DELAY_SECONDS = 0.5
RADIUS = 3
COLORS = ["#ff0000", "#00ff00", "#0000ff", "#9c9c9c", "#3cbcbc"]

def get_colors(x, y):
    return ["#%02x%02x%02x" % (int(float(red / xDim) * 255), int(float(green / yDim) * 255), 150) for red, green in zip(x, y)]

    

In [4]:
# grid = np.array([[0,0,0, 2],[100,100,0, 3]])
grid = np.array([[0,0,0,0]])

TOOLS="hover,crosshair,pan,reset,tap,save,box_select,poly_select"

distance = grid[:,0]
fair = grid[:,1]
radii = grid[:,2]
clusters = grid[:,3]
colors = c = [COLORS[color_index] for color_index in clusters]
colors = ['#ffffff']

size = grid.shape[0] # amount of data points
print('shape', grid.shape)
print(size)

p = figure(title="Optimized Visaulization Pipeline: Taxi Data", width=800, height=800, tools=TOOLS,
          x_axis_label='distance', y_axis_label='fair amount')
# ["#%02x%02x%02x" % (int((red / xDim) * 255), int((green / yDim) * 255), 150) for red, green in zip(50+2*distance, 30+2*fair)]
# colors = get_colors(distance, fair)
# fill_color=colors
scat = p.scatter(x = distance, y = fair, radius=1, fill_color=colors, fill_alpha=0.8, line_color=None)

shape (1, 4)
1


In [5]:
# functions

def getData(x=100,y=100):
    relativePath = "{}{}?x={}&y={}".format(basePath,"scatter",x,y)
    conn = http.client.HTTPConnection(baseEndpoint)
    conn.request("GET", relativePath)
    r1 = conn.getresponse()

    if (r1.status >= 400):
        print("Error:", relativePath, r1.status, r1.reason)
        return []
    else:
        payload = r1.read()
        jsonPayload = payload.decode('utf-8')
        return json.loads(jsonPayload)

def initStream(x=100, y=100, f="KMEANSVDDA"):
    relativePath = "{}{}?x={}&y={}&f={}".format(basePath, "stream/cluster",x,y,f)
    conn = http.client.HTTPConnection(baseEndpoint)
    conn.request("GET", relativePath)
    r1 = conn.getresponse()

    if (r1.status >= 400):
        print("Error:", relativePath, r1.status, r1.reason)

def reset_plot():
    scat.data_source.data['x'] = []
    scat.data_source.data['y'] = []
    scat.data_source.data['fill_color'] = np.array([])
    time.sleep(1)
    push_notebook()
    
def test_scatter_plot():
    xDim = 500
    yDim = 500
    data = getData(xDim,yDim)

    grid = np.array(data)
    size = grid.shape[0]
    x = grid[:,0]
    y = grid[:,1]
    size = grid.shape[0]
    scat.data_source.data['x'] = x
    scat.data_source.data['y'] = y
    scat.data_source.data['fill_color'] = ["#000000" for color in x]

    time.sleep(MESSAGE_DELAY_SECONDS)
    push_notebook()

    data = [[0,0,0], [1000,1000,0]]
    
def consumeQueue(ch, method, properties, body):
#     print(" [x] Consuming Queue Message:%r" % (method.routing_key))
    
    # parse message; e.g. {"data":[[1,2,15],[4,2,162], ... [23,43,10]]}
    jsonPayload = body.decode('utf-8')
    data = json.loads(jsonPayload)
    
#     acknowledge message received
#     ch.basic_ack(delivery_tag = method.delivery_tag)
    
    # convert data
    grid = np.array(data)
    x = grid[:,0]
    y = grid[:,1]
    clusters = grid[:,2]
    c = [COLORS[color_index] for color_index in clusters]
#     print(clusters)
    
#     radii = grid[:,2]
    # #colors = ["#%02x%02x%02x" % (int(red % 255), int(green % 255), 150) for red, green in zip(50+2*x, 30+2*y)]
    ## ["#%02x%02x%02x" % (int(float(red / xDim) * 150), int(float(green / yDim) * 150), 150) for red, green in zip(distance, fair)]
#     colors = get_colors(x, y)
    
    # update plot
#     x = np.concatenate((scat.data_source.data['x'], x))
#     y = np.concatenate((scat.data_source.data['y'], y))
#     print (x)
#     print (y)
    
#     scat.data_source.data['x'] = x
    scat.data_source.data['x'] = np.concatenate((scat.data_source.data['y'], x))
#     scat.data_source.data['y'] = y
    scat.data_source.data['y'] = np.concatenate((scat.data_source.data['y'], y))
    scat.data_source.data['fill_color'] = np.concatenate((scat.data_source.data['fill_color'], c))
#     scat.data_source.data['radius'] = [(radius % RADIUS) for radius in radii]
#     scat.data_source.data['fill_color'] = colors
    
    time.sleep(MESSAGE_DELAY_SECONDS)
    push_notebook()

In [36]:
# for plots read: http://bokeh.pydata.org/en/latest/docs/user_guide/notebook.html

connection = pika.BlockingConnection(parameters)
channel = connection.channel()


In [37]:
#run = 1
#
#def consume():
#    while(run):
#        channel.basic_consume(consumeQueue, queue='BDAPRO', no_ack=True)
#        time.sleep(MESSAGE_DELAY_SECONDS)

channel.basic_consume(consumeQueue, queue='BDAPRO2', no_ack=True)        
channel.start_consuming()

connection.close()

KeyboardInterrupt: 

In [34]:
connection.close()

In [9]:
reset_plot()

In [8]:
test_scatter_plot()

In [ ]:
xDim = 1000
yDim = 1000
initStream(xDim,yDim, "KMEANSVDDA") # KMEANS | KMEANSVDDA

In [7]:
show(p, notebook_handle=True)